In [119]:
import pandas as pd
import json
import numpy as np
from tqdm import tqdm
import requests
import traceback
import gcsfs

from kloppy import skillcorner

In [137]:
fs = gcsfs.GCSFileSystem(token="google_default")

In [136]:
BUCKET_NAME = "arthur_tmp"
BASE_GCS_PATH = f"gs://{BUCKET_NAME}/analytics_cup"
PROCESSED_DIR = f"{BASE_GCS_PATH}/processed"

X_MIN, X_MAX = -52, 52
Y_MIN, Y_MAX = -34, 34
PITCH_LENGTH, PITCH_WIDTH = 105.0, 68.0

params = {
    'v_max': 5.0,
    'a_max': 7.0,
    'reaction_time': 0.7,
    'lambda_param': 3.0,
    'time_max': 6.0,
    'dt': 0.02
}

In [ ]:
def read_parquet_gcs(fs: gcsfs.GCSFileSystem, gcs_path: str) -> pd.DataFrame:
    if not fs.exists(gcs_path):
        raise FileNotFoundError(gcs_path)
    # open as file-like and let pandas/pyarrow read
    with fs.open(gcs_path, "rb") as f:
        return pd.read_parquet(f)
    
def write_parquet_gcs(df: pd.DataFrame, gcs_path: str):
    # rely on pandas+fsspec to write directly
    # storage_options may be required depending on environment (GCE default works)
    df.to_parquet(gcs_path, index=False)

# DATA PREP

In [69]:
def load_tracking_full_from_kloppy(
    match_id: int,
    sort_rows: bool = False,
    require_ball_detected: bool = True,
) -> pd.DataFrame:

    dataset = skillcorner.load_open_data(
        match_id=match_id,
        coordinates="skillcorner",
    )

    rows = []

    for frame in dataset.frames:
        frame_id = frame.frame_id
        ts = frame.timestamp
        period = frame.period.id if frame.period is not None else None

        # --- Players ---
        for player, coords in frame.players_coordinates.items():
            is_detected = coords is not None

            rows.append({
                "match_id": match_id,
                "time": ts,
                "frame": frame_id,
                "period": period,
                "player_id": player.player_id,
                "is_detected": is_detected,
                "is_ball": False,
                "x": coords.x if is_detected else np.nan,
                "y": coords.y if is_detected else np.nan,
            })

        # --- Ball ---
        ball_coords = frame.ball_coordinates
        ball_detected = ball_coords is not None

        if (not require_ball_detected) or ball_detected:
            rows.append({
                "match_id": match_id,
                "time": ts,
                "frame": frame_id,
                "period": period,
                "player_id": -1,
                "is_detected": ball_detected,
                "is_ball": True,
                "x": ball_coords.x if ball_detected else np.nan,
                "y": ball_coords.y if ball_detected else np.nan,
            })

    df = pd.DataFrame(rows)
    if df.empty:
        return df

    # --- Types ---
    df["is_ball"] = df["is_ball"].astype(bool)
    df["is_detected"] = df["is_detected"].astype(bool)
    df["frame"] = df["frame"].astype(int)
    df["player_id"] = df["player_id"].astype(int)

    # --- Rescaling ---
    den_x, den_y = (X_MAX - X_MIN), (Y_MAX - Y_MIN)
    valid_xy = df[["x", "y"]].notna().all(axis=1)

    df["x_rescaled"] = np.where(
        valid_xy,
        (df["x"] - X_MIN) / den_x * PITCH_LENGTH,
        np.nan
    )
    df["y_rescaled"] = np.where(
        valid_xy,
        (df["y"] - Y_MIN) / den_y * PITCH_WIDTH,
        np.nan
    )

    # --- Sorting ---
    if sort_rows:
        df = df.sort_values(["frame", "is_ball", "player_id"]).reset_index(drop=True)
    else:
        df = df.sort_values(["frame"]).reset_index(drop=True)

    print(
        f"[match {match_id}] rows: {len(df)}, "
        f"players: {(~df['is_ball']).sum()}, "
        f"ball: {df['is_ball'].sum()}"
    )

    return df

In [70]:
df = load_tracking_full_from_kloppy(match_id=1886347)

[match 1886347] rows: 724617, players: 693106, ball: 31511


In [71]:
df.head()

,match_id,time,frame,period,player_id,is_detected,is_ball,x,y,x_rescaled,y_rescaled
0,1886347,0 days 00:00:01.800000,28,1,51009,True,False,-40.71,-0.27,11.398558,33.73
1,1886347,0 days 00:00:01.800000,28,1,-1,True,True,0.03,-0.36,52.530288,33.64
2,1886347,0 days 00:00:01.800000,28,1,38673,True,False,1.76,9.23,54.276923,43.23
3,1886347,0 days 00:00:01.800000,28,1,50951,True,False,-0.07,-8.13,52.429327,25.87
4,1886347,0 days 00:00:01.800000,28,1,965685,True,False,7.63,-17.23,60.203365,16.77


In [29]:
def load_meta_mapping_from_kloppy(match_id: int) -> pd.DataFrame:
    dataset = skillcorner.load_open_data(
        match_id=match_id,
        coordinates="skillcorner",
    )

    teams = dataset.metadata.teams  # home_team, away_team
    if len(teams) != 2:
        return pd.DataFrame()

    home_team, away_team = teams

    rows = []

    for team, team_type in [(home_team, "home"), (away_team, "away")]:
        for player in team.players:
            rows.append({
                "match_id": match_id,
                "player_id": player.player_id,  # ou player.jersey_no
                "team_id": team.team_id,
                "team_name": team.name,
                #"team_short": team.short_name,
                "player_short_name": player.name,  # ou player.short_name
                "player_role": getattr(player, "position", None).name if getattr(player, "position", None) else None,
                "position_group": getattr(player, "position", None).position_group if getattr(player, "position", None) else None,
                "home_team_id": home_team.team_id,
                "away_team_id": away_team.team_id,
            })

    return pd.DataFrame(rows)

In [40]:
def load_meta_from_github(match_id: int) -> pd.DataFrame:
    """
    Charge le fichier meta SkillCorner depuis GitHub et le transforme
    dans le même format que load_meta_mapping (ton ancien loader).
    """

    url = (
        "https://raw.githubusercontent.com/SkillCorner/opendata/master/"
        f"data/matches/{match_id}/{match_id}_match.json"
    )

    r = requests.get(url)
    if r.status_code != 200:
        # Pas de fichier trouvé
        return pd.DataFrame()

    meta = r.json()
    if not meta:
        return pd.DataFrame()

    # On récupère les infos d'équipe
    home_team = meta.get("home_team", {}) or {}
    away_team = meta.get("away_team", {}) or {}

    rows = []
    for p in meta.get("players", []):
        # Si pas d'id, on skip
        if "id" not in p:
            continue

        team_id = p.get("team_id")
        team = home_team if team_id == home_team.get("id") else away_team

        rows.append({
            "match_id": meta.get("id"),
            "player_id": p.get("id"),
            "team_id": team_id,
            "team_name": team.get("name"),
            "team_short": team.get("short_name"),
            "player_short_name": p.get("short_name"),
            "player_role": p.get("player_role", {}).get("name"),
            "position_group": p.get("player_role", {}).get("position_group"),
            "home_team_id": home_team.get("id"),
            "away_team_id": away_team.get("id"),
        })

    return pd.DataFrame(rows)

In [42]:
df_meta = load_meta_from_github(match_id=1886347)

In [62]:
df_meta

,match_id,player_id,team_id,team_name,team_short,player_short_name,player_role,position_group,home_team_id,away_team_id
0,1886347,38673,4177,Auckland FC,Auckland FC,G. May,Center Forward,Center Forward,4177,1805
1,1886347,51713,4177,Auckland FC,Auckland FC,C. Elliott,Right Back,Full Back,4177,1805
2,1886347,50951,4177,Auckland FC,Auckland FC,J. Brimmer,Center Forward,Center Forward,4177,1805
3,1886347,50978,1805,Newcastle United Jets FC,Newcastle,C. Timmins,Left Defensive Midfield,Midfield,4177,1805
4,1886347,133498,4177,Auckland FC,Auckland FC,F. De Vries,Left Back,Full Back,4177,1805
5,1886347,33697,4177,Auckland FC,Auckland FC,N. Pijnaker,Left Center Back,Central Defender,4177,1805
6,1886347,51667,4177,Auckland FC,Auckland FC,D. Hall,Right Center Back,Central Defender,4177,1805
7,1886347,14736,4177,Auckland FC,Auckland FC,L. Verstraete,Defensive Midfield,Midfield,4177,1805
8,1886347,966125,1805,Newcastle United Jets FC,Newcastle,W. Will Dobson,Substitute,Other,4177,1805
9,1886347,966124,1805,Newcastle United Jets FC,Newcastle,N. James,Substitute,Other,4177,1805


In [46]:
def load_events_basic_from_github(match_id: int) -> pd.DataFrame:
    url = (
        "https://raw.githubusercontent.com/SkillCorner/opendata/master/"
        f"data/matches/{match_id}/{match_id}_dynamic_events.csv"
    )

    df = pd.read_csv(url)

    # --- même logique que ton code existant ---
    if "event_type" not in df.columns or "end_type" not in df.columns:
        return pd.DataFrame()

    #df = df.query(
    #    "event_type == 'player_possession' and end_type == 'pass'"
    #).copy()

    if df.empty:
        return pd.DataFrame()

    df["player_in_possession_id"] = df["player_id"]

    # rescale SkillCorner -> 105x68
    X_MIN, X_MAX = -52, 52
    Y_MIN, Y_MAX = -34, 34

    if {"x_start", "y_start", "x_end", "y_end"}.issubset(df.columns):
        df["x_start_rescaled"] = (df["x_start"] - X_MIN) / (X_MAX - X_MIN) * 105.0
        df["y_start_rescaled"] = (df["y_start"] - Y_MIN) / (Y_MAX - Y_MIN) * 68.0
        df["x_end_rescaled"]   = (df["x_end"]   - X_MIN) / (X_MAX - X_MIN) * 105.0
        df["y_end_rescaled"]   = (df["y_end"]   - Y_MIN) / (Y_MAX - Y_MIN) * 68.0

    df = df.loc[:, df.notna().any(axis=0)]

    if "frame_start" in df.columns:
        df = df.sort_values("frame_start").reset_index(drop=True)

    return df

In [47]:
df_events = load_events_basic_from_github(match_id=1886347)

In [48]:
df_events

,event_id,index,match_id,frame_start,frame_end,frame_physical_start,time_start,time_end,minute_start,second_start,...,xshot_player_possession_max,is_player_possession_start_matched,is_player_possession_end_matched,is_previous_pass_matched,is_pass_reception_matched,fully_extrapolated,x_start_rescaled,y_start_rescaled,x_end_rescaled,y_end_rescaled
0,8_0,0,1886347,28,28,NaN,00:01.8,00:01.8,0,1,...,NaN,True,True,NaN,True,False,53.237019,34.49,53.237019,34.49
1,8_1,1,1886347,48,58,NaN,00:03.8,00:04.8,0,3,...,NaN,True,True,True,True,False,29.975481,35.22,30.076442,36.70
2,7_0,2,1886347,48,53,NaN,00:03.8,00:04.3,0,3,...,NaN,True,True,NaN,NaN,False,41.929327,31.22,41.030769,32.62
3,7_1,3,1886347,48,58,NaN,00:03.8,00:04.8,0,3,...,NaN,True,True,NaN,True,False,31.611058,50.66,31.944231,51.69
4,9_0,4,1886347,56,58,34.0,00:02.4,00:04.8,0,2,...,NaN,True,True,True,True,NaN,65.655288,34.06,66.614423,33.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5074,7_2541,5074,1886347,58757,58761,NaN,96:35.7,96:36.1,96,35,...,NaN,True,True,NaN,NaN,False,84.050481,42.32,84.191827,43.17
5075,9_936,5075,1886347,58760,58761,58760.0,96:36.0,96:36.1,96,36,...,0.402,True,True,True,NaN,NaN,30.561058,30.32,30.520673,30.26
5076,7_2542,5077,1886347,58907,58932,NaN,96:50.7,96:53.2,96,50,...,NaN,True,True,NaN,NaN,True,50.904808,7.32,52.005288,4.07
5077,8_998,5076,1886347,58907,58932,NaN,96:50.7,96:53.2,96,50,...,NaN,True,True,NaN,NaN,False,11.751923,21.77,14.407212,15.52


In [106]:
df_events.event_type.value_counts()

event_type
passing_option        2544
player_possession      999
on_ball_engagement     937
off_ball_run           599
Name: count, dtype: int64

In [107]:
df_events.end_type.value_counts()

end_type
pass                   902
indirect_regain        120
possession_loss         48
indirect_disruption     44
direct_regain           24
shot                    23
direct_disruption       18
foul_suffered           15
foul_committed          13
clearance                9
unknown                  2
Name: count, dtype: int64

In [49]:
df_events.team_out_of_possession_phase_type.value_counts()

team_out_of_possession_phase_type
medium_block             2112
low_block                1250
high_block                795
chaotic                   409
defending_set_play        225
defending_direct          136
defending_transition       59
defending_quick_break      47
disruption                 46
Name: count, dtype: int64

In [50]:
df_events.team_in_possession_phase_type.value_counts()

team_in_possession_phase_type
create         2112
finish         1250
build_up        795
chaotic         409
set_play        225
direct          136
transition       59
quick_break      47
disruption       46
Name: count, dtype: int64

In [51]:
def to_seconds(series: pd.Series) -> pd.Series:
    """
    Convert time to seconds.
    Handles strings like 'HH:MM:SS.ff' (e.g., '00:00:00.10'),
    numeric seconds, and large integers/floats in ms.
    Returns a float Series.
    """
    # if already numeric-ish, start there
    s_num = pd.to_numeric(series, errors="coerce")

    # try timedelta parsing for strings like HH:MM:SS.ff
    s_td = pd.to_timedelta(series.astype(str).str.strip().str.replace(",", ".", regex=False),
                           errors="coerce")
    s_sec_td = s_td.dt.total_seconds()

    # prefer timedelta parse when available, else numeric
    s = s_sec_td.fillna(s_num)

    # if values look like milliseconds (very large), convert to seconds
    if s.notna().any():
        median_val = s.dropna().median()
        # heuristic: median > 1e6 suggests ms ticks (e.g., 1,650,000,000)
        if median_val is not None and median_val > 1e6:
            s = s / 1000.0

    return s.astype(float)

In [52]:
def estimate_fps_from_time_df(df: pd.DataFrame, group_cols=("match_id",)) -> float:
    """
    Estimate frames-per-second from median time delta.
    Works even if you pass only ball rows or all rows.
    """
    t = to_seconds(df["time"])
    # compute dt within each group (match)
    if not isinstance(group_cols, (list, tuple)):
        group_cols = (group_cols,)
    dt = t.groupby([df[c] for c in group_cols]).diff().astype(float)
    dt = dt[(dt > 0) & dt.notna()]
    if dt.empty:
        return 25.0
    fps = 1.0 / float(dt.median())
    return float(np.clip(fps, 5, 120))

In [53]:
def compute_all_velocities(
    tr_df: pd.DataFrame,
    fps_fallback: float = 10.0,
    interp_gap_s: float = 0.40,
    smooth_window_s: float = 0.70,
    clip_speed_player: float = 7.0,   # tighter and realistic
    clip_speed_ball: float = 35.0,    # typical max shot
) -> pd.DataFrame:
    """
    Compute velocities (vx_mps, vy_mps, speed_mps) for all entities (players + ball),
    with physical clipping and gap-safe smoothing.
    """

    df = tr_df.copy()
    df["time_s"] = to_seconds(df["time"])
    df = df.sort_values(["match_id", "is_ball", "player_id", "frame"]).reset_index(drop=True)

    results = []

    for (match_id, is_ball, pid), g in df.groupby(["match_id", "is_ball", "player_id"], sort=False):
        g = g.sort_values("frame").copy()
        if len(g) < 3:
            continue

        # Convert coords
        x = pd.to_numeric(g["x_rescaled"], errors="coerce")
        y = pd.to_numeric(g["y_rescaled"], errors="coerce")
        t = pd.to_numeric(g["time_s"], errors="coerce")

        # fallback if missing time
        if t.isna().any():
            dt_nominal = 1.0 / fps_fallback
            t = np.arange(len(x)) * dt_nominal

        dt = t.diff()
        dt_mean = np.nanmedian(dt)
        dt[dt <= 0] = dt_mean  # fix zero or negative steps

        # remove unrealistic time jumps
        mask_big_jump = dt > 3 * dt_mean
        if mask_big_jump.any():
            x[mask_big_jump] = np.nan
            y[mask_big_jump] = np.nan

        # interpolate + smooth
        x_i = x.interpolate(limit_direction="both", limit=3)
        y_i = y.interpolate(limit_direction="both", limit=3)
        window_frames = int(round(smooth_window_s / (dt_mean or (1.0 / fps_fallback))))
        if window_frames % 2 == 0:
            window_frames += 1
        x_s = x_i.rolling(window_frames, center=True, min_periods=1).median()
        y_s = y_i.rolling(window_frames, center=True, min_periods=1).median()

        # compute velocities
        vx = x_s.diff() / dt
        vy = y_s.diff() / dt

        # smooth velocity slightly
        vx = vx.rolling(3, center=True, min_periods=1).mean()
        vy = vy.rolling(3, center=True, min_periods=1).mean()

        # clip velocities physically
        vmax = clip_speed_ball if is_ball else clip_speed_player
        vx = vx.clip(lower=-vmax, upper=vmax)
        vy = vy.clip(lower=-vmax, upper=vmax)
        speed = np.sqrt(vx**2 + vy**2)

        g["vx_mps"] = vx
        g["vy_mps"] = vy
        g["speed_mps"] = speed
        results.append(g)

    df_vel = pd.concat(results, ignore_index=True)
    return df_vel


In [115]:
def merge_tracking_and_events(match_id: int, df_events: pd.DataFrame, df_tracking: pd.DataFrame) -> pd.DataFrame:
    """
    Merge tracking data with event info so each pass start frame inherits:
      - team_in_possession
      - attacking_side
      - player_in_possession_id
      - pass outcome + coordinates (rescaled)
      - labels such as lead_to_goal if present
    
    Only uses the *start frame* of each pass event, since SoccerMap learns from
    the player/ball configuration before the pass.
    """
    frames_all = []

    # ✅ Keep only pass possessions (successful / unsuccessful)
    df_events_pass = df_events.query(
        "event_type == 'player_possession'" #and end_type == 'pass'"
    ).copy()

    if df_events_pass.empty:
        print(f"⚠️ No valid pass events for match {match_id}")
        return pd.DataFrame()

    for _, ev in tqdm(df_events_pass.iterrows(), total=len(df_events_pass), desc=f"Merging match {match_id}"):
        frame_start = int(ev["frame_start"]) if "frame_start" in ev else None
        if frame_start is None or frame_start not in df_tracking["frame"].values:
            continue

        # take only the frame at the start of the pass
        tr_slice = df_tracking.loc[df_tracking["frame"] == frame_start].copy()
        if tr_slice.empty:
            continue

        # Assign event info (broadcast to all players + ball in that frame)
        tr_slice["event_id"] = ev.get("event_id")
        tr_slice["event_type"] = ev.get("event_type")
        tr_slice["end_type"] = ev.get("end_type")
        tr_slice["pass_outcome"] = ev.get("pass_outcome", None)
        tr_slice["team_in_possession_phase_type"] = ev.get("team_in_possession_phase_type", None)
        tr_slice["team_out_of_possession_phase_type"] = ev.get("team_out_of_possession_phase_type", None)
        tr_slice["player_in_possession_id"] = ev.get("player_in_possession_id", ev.get("player_id", None))
        tr_slice["team_in_possession"] = ev.get("team_id")
        tr_slice["team_in_possession_shortname"] = ev.get("team_shortname")
        tr_slice["attacking_side"] = ev.get("attacking_side")
        tr_slice["period"] = ev.get("period")

        # --- Pass start and end coordinates ---
        for c in ["x_start_rescaled", "y_start_rescaled", "x_end_rescaled", "y_end_rescaled"]:
            if c in ev:
                tr_slice[c] = ev[c]

        # --- Add label columns if available (e.g. for model training) ---
        for label_col in ["lead_to_goal", "lead_to_shot", "lead_to_box_entry"]:
            if label_col in ev:
                tr_slice[label_col] = ev[label_col]

        frames_all.append(tr_slice)

    if not frames_all:
        print(f"⚠️ No matching frames for match {match_id}")
        return pd.DataFrame()

    df_merged = pd.concat(frames_all, ignore_index=True)

    # --- Clean up ---
    df_merged["match_id"] = match_id
    df_merged = df_merged.sort_values(["period", "frame"]).reset_index(drop=True)

    #print(f"✅ [match {match_id}] merged frames: {len(df_merged):,}")
    return df_merged

In [116]:
def normalize_tracking_direction(df_merged: pd.DataFrame) -> pd.DataFrame:
    """
    Flip coordinates so that team_in_possession always attacks left -> right.
    """
    df_norm = df_merged.copy()

    mask = df_norm["attacking_side"] == "right_to_left"
    df_norm.loc[mask, "x_rescaled"] = 105.0 - df_norm.loc[mask, "x_rescaled"]
    df_norm.loc[mask, "y_rescaled"] = 68.0 - df_norm.loc[mask, "y_rescaled"]
    df_norm.loc[mask, "vx_mps"] = -df_norm.loc[mask, "vx_mps"]
    df_norm.loc[mask, "vy_mps"] = -df_norm.loc[mask, "vy_mps"]

    return df_norm

In [120]:
def process_data(match_id: int, save_to_gcs=False) -> int:
    try:
        if save_to_gcs:
            outpath = f"{PROCESSED_DIR}/{match_id}_normed.parquet"
            if fs.exists(outpath):
                print(f"{match_id} already processed")
                return match_id

        # 1) load base data
        tr = load_tracking_full_from_kloppy(match_id)
        ev = load_events_basic_from_github(match_id)
        meta = load_meta_from_github(match_id)

        if tr is None or ev is None or meta is None or tr.empty or ev.empty or meta.empty:
            # skip if any missing
            # print(f"⚠️ Skipped {match_id} (missing one of: tracking, events, meta)")
            return match_id

        # 2) compute velocities (user function)
        tr = compute_all_velocities(tr)

        # 3) add meta
        tr = tr.merge(meta, on=["match_id", "player_id"], how="left", validate="m:1")
        tr.loc[tr["is_ball"], ["team_id", "team_name", "team_short"]] = np.nan

        # 4) merge with events
        df_merged = merge_tracking_and_events(match_id, ev, tr)

        # 5) normalize direction
        df_normed = normalize_tracking_direction(df_merged)

        # 6) sanity & write
        if df_normed[["x_rescaled", "y_rescaled"]].isna().all().any():
            raise ValueError("Missing rescaled coordinates after normalization.")
        df_normed = df_normed.dropna(subset=['x_rescaled', 'y_rescaled'])
        
        # 7) write to GCS
        write_parquet_gcs(df_normed, outpath)
        
        return match_id

    except Exception as e:
        # log error but don't crash the pool
        print(f"❌ Error processing {match_id}: {e}")
        traceback.print_exc()
        return match_id

In [118]:
%%time

process_data(match_id=1886347, save_to_gcs=True)

[match 1886347] rows: 724617, players: 693106, ball: 31511


Merging match 1886347: 100%|██████████| 999/999 [00:07<00:00, 127.71it/s]


CPU times: user 32.6 s, sys: 286 ms, total: 32.9 s
Wall time: 37.1 s


In [122]:
match_id=1886347
df_normed = read_parquet_gcs(fs, gcs_path=f"{PROCESSED_DIR}/{match_id}_normed.parquet")

In [152]:
df_normed.team_in_possession

0        1805
1        1805
2        1805
3        1805
4        1805
         ... 
22461    4177
22462    4177
22463    4177
22464    4177
22465    4177
Name: team_in_possession, Length: 22466, dtype: int64

In [179]:
df_normed[df_normed.frame==28]

,match_id,time,frame,period,player_id,is_detected,is_ball,x,y,x_rescaled,...,player_in_possession_id,team_in_possession,team_in_possession_shortname,attacking_side,x_start_rescaled,y_start_rescaled,x_end_rescaled,y_end_rescaled,lead_to_goal,lead_to_shot
0,1886347,0 days 00:00:01.800000,28,1,14736,True,False,9.73,6.44,62.323558,...,966120,1805,Newcastle,left_to_right,53.237019,34.49,53.237019,34.49,False,False
1,1886347,0 days 00:00:01.800000,28,1,23418,True,False,9.03,-2.78,61.616827,...,966120,1805,Newcastle,left_to_right,53.237019,34.49,53.237019,34.49,False,False
2,1886347,0 days 00:00:01.800000,28,1,33697,True,False,17.43,-3.57,70.097596,...,966120,1805,Newcastle,left_to_right,53.237019,34.49,53.237019,34.49,False,False
3,1886347,0 days 00:00:01.800000,28,1,38673,True,False,1.76,9.23,54.276923,...,966120,1805,Newcastle,left_to_right,53.237019,34.49,53.237019,34.49,False,False
4,1886347,0 days 00:00:01.800000,28,1,50951,True,False,-0.07,-8.13,52.429327,...,966120,1805,Newcastle,left_to_right,53.237019,34.49,53.237019,34.49,False,False
5,1886347,0 days 00:00:01.800000,28,1,50978,True,False,-6.52,6.95,45.917308,...,966120,1805,Newcastle,left_to_right,53.237019,34.49,53.237019,34.49,False,False
6,1886347,0 days 00:00:01.800000,28,1,50983,True,False,-0.64,-33.03,51.853846,...,966120,1805,Newcastle,left_to_right,53.237019,34.49,53.237019,34.49,False,False
7,1886347,0 days 00:00:01.800000,28,1,51009,True,False,-40.71,-0.27,11.398558,...,966120,1805,Newcastle,left_to_right,53.237019,34.49,53.237019,34.49,False,False
8,1886347,0 days 00:00:01.800000,28,1,51649,True,False,-21.72,0.22,30.571154,...,966120,1805,Newcastle,left_to_right,53.237019,34.49,53.237019,34.49,False,False
9,1886347,0 days 00:00:01.800000,28,1,51667,True,False,17.93,4.82,70.602404,...,966120,1805,Newcastle,left_to_right,53.237019,34.49,53.237019,34.49,False,False


# PITCH CONTROL

In [132]:
from sklearn.cluster import AgglomerativeClustering

In [161]:
def get_player_id_position(tracking_df, player_position: str) -> dict:
    """
    Retourne les IDs des joueurs occupant une position donnée pour chaque équipe.
    Si aucun joueur n'est trouvé pour une équipe, la valeur correspondante est None.
    """
    home_team_id = tracking_df["home_team_id"].iloc[0]
    away_team_id = tracking_df["away_team_id"].iloc[0]

    def get_player_id(team_id):
        filtered = tracking_df[
            (tracking_df["position_group"] == player_position)
            & (tracking_df["team_id"] == team_id)
        ]
        return filtered["player_id"].iloc[0] if not filtered.empty else None

    return {
        home_team_id: get_player_id(home_team_id),
        away_team_id: get_player_id(away_team_id),
    }

In [187]:
player_id_position_dict = get_player_id_position(df_normed, "Center Forward")
player_id_position_dict

{4177.0: 38673, 1805.0: 966120}

In [162]:
def build_team_tracking(df, team_side='home'):
    """
    Construit un format similaire à Metrica : une ligne = un frame,
    avec les positions et vitesses de chaque joueur de l'équipe.
    """
    team_id = df[f'{team_side}_team_id'].iloc[0]
    df_team = df[df['team_id'] == team_id].copy()
    team_name = df_team['team_name'].iloc[0]
    
    # Liste des joueurs
    players = df_team['player_id'].unique()
    
    tracking = (
        df_team
        .pivot(index='frame', columns='player_id', values=['x_rescaled', 'y_rescaled'])
        .sort_index()
    )
    tracking.columns = [f"{c[0]}_{int(c[1])}" for c in tracking.columns]
    tracking = tracking.reset_index()

    # Ajoute ball data
    ball = (
        df[df['is_ball']]
        .set_index('frame')[['x_rescaled', 'y_rescaled']]
        .rename(columns={'x_rescaled': 'ball_x', 'y_rescaled': 'ball_y'})
    )
    tracking = tracking.merge(ball, on='frame', how='left')
    
    tracking['player_id_in_possession'] = df.groupby('frame')['player_in_possession_id'].first().values
    tracking['Period'] = df.groupby('frame')['period'].first().values
    tracking['Time [s]'] = df.groupby('frame')['time'].first().values
    tracking['team_name'] = team_name
    
    return tracking, players

In [188]:
tracking, players = build_team_tracking(df_normed, team_side='home')  

In [189]:
tracking

,frame,x_rescaled_14736,x_rescaled_23418,x_rescaled_31147,x_rescaled_33697,x_rescaled_38673,x_rescaled_43829,x_rescaled_50951,x_rescaled_51667,x_rescaled_51713,...,y_rescaled_133501,y_rescaled_163972,y_rescaled_285188,y_rescaled_965685,ball_x,ball_y,player_id_in_possession,Period,Time [s],team_name
0,28,62.323558,61.616827,NaN,70.097596,54.276923,NaN,52.429327,70.602404,69.188942,...,51.14,NaN,34.14,16.77,52.530288,33.64,966120,1,0 days 00:00:01.800000,Auckland FC
1,48,59.900481,59.284615,NaN,70.844712,47.542788,NaN,43.655769,69.330288,67.311058,...,49.50,NaN,33.99,14.22,31.035577,34.84,51649,1,0 days 00:00:03.800000,Auckland FC
2,72,56.003365,61.142308,NaN,70.915385,36.386538,NaN,36.073558,71.702885,70.077404,...,51.51,NaN,35.15,20.13,35.205288,52.20,735578,1,0 days 00:00:06.200000,Auckland FC
3,202,40.737981,40.717788,NaN,24.392308,60.637500,NaN,52.065865,27.673558,38.981250,...,8.01,NaN,34.21,52.87,24.765865,40.47,33697,1,0 days 00:00:19.200000,Auckland FC
4,243,44.392788,42.959135,NaN,28.925481,62.162019,NaN,54.428365,27.956250,40.192788,...,16.69,NaN,38.02,55.06,42.757212,65.38,133498,1,0 days 00:00:23.300000,Auckland FC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
972,58581,73.893750,69.411058,81.576923,82.425000,NaN,72.005769,NaN,80.476442,79.466827,...,NaN,33.30,33.44,12.98,64.746635,9.43,735574,2,0 days 00:51:18.100000,Auckland FC
973,58602,77.013462,77.306250,85.009615,85.403365,NaN,73.096154,NaN,83.061058,82.142308,...,NaN,30.30,32.69,7.53,78.517788,1.85,50983,2,0 days 00:51:20.200000,Auckland FC
974,58710,85.150962,84.333173,89.078365,90.148558,NaN,77.770673,NaN,87.563942,88.391827,...,NaN,15.84,32.56,4.38,67.391827,4.29,735574,2,0 days 00:51:31,Auckland FC
975,58733,81.920192,78.033173,84.787500,86.130288,NaN,74.893269,NaN,85.262019,85.887981,...,NaN,20.23,33.31,8.73,60.405288,27.43,51649,2,0 days 00:51:33.300000,Auckland FC


In [190]:
players

array([ 14736,  23418,  33697,  38673,  50951,  51667,  51713, 133498,
       133501, 285188, 965685,  43829, 163972,  31147])

In [163]:
def calculate_player_velocities(tracking, players, frame_rate=4):
    """
    Calcule vx, vy pour chaque joueur (en m/s)
    """
    for pid in players:
        for axis in ['x', 'y']:
            col = f"{axis}_rescaled_{pid}"
            vcol = f"v{axis}_{pid}"
            tracking[vcol] = tracking[col].diff() * frame_rate
    return tracking

In [191]:
tr_sp = calculate_player_velocities(tracking, players)

In [192]:
tr_sp

,frame,x_rescaled_14736,x_rescaled_23418,x_rescaled_31147,x_rescaled_33697,x_rescaled_38673,x_rescaled_43829,x_rescaled_50951,x_rescaled_51667,x_rescaled_51713,...,vx_285188,vy_285188,vx_965685,vy_965685,vx_43829,vy_43829,vx_163972,vy_163972,vx_31147,vy_31147
0,28,62.323558,61.616827,NaN,70.097596,54.276923,NaN,52.429327,70.602404,69.188942,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,48,59.900481,59.284615,NaN,70.844712,47.542788,NaN,43.655769,69.330288,67.311058,...,-1.898077,-0.60,0.282692,-10.20,NaN,NaN,NaN,NaN,NaN,NaN
2,72,56.003365,61.142308,NaN,70.915385,36.386538,NaN,36.073558,71.702885,70.077404,...,-2.342308,4.64,-10.661538,23.64,NaN,NaN,NaN,NaN,NaN,NaN
3,202,40.737981,40.717788,NaN,24.392308,60.637500,NaN,52.065865,27.673558,38.981250,...,-330.951923,-3.76,-9.167308,130.96,NaN,NaN,NaN,NaN,NaN,NaN
4,243,44.392788,42.959135,NaN,28.925481,62.162019,NaN,54.428365,27.956250,40.192788,...,-1.857692,15.24,-9.571154,8.76,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
972,58581,73.893750,69.411058,81.576923,82.425000,NaN,72.005769,NaN,80.476442,79.466827,...,-0.484615,-0.80,-6.825000,-10.68,-2.059615,-4.24,-9.975000,-9.52,7.875000,-0.36
973,58602,77.013462,77.306250,85.009615,85.403365,NaN,73.096154,NaN,83.061058,82.142308,...,4.563462,-3.00,9.975000,-21.80,4.361538,-7.24,-1.776923,-12.00,13.730769,-13.84
974,58710,85.150962,84.333173,89.078365,90.148558,NaN,77.770673,NaN,87.563942,88.391827,...,6.057692,-0.52,31.661538,-12.60,18.698077,-32.76,25.321154,-57.84,16.275000,-7.56
975,58733,81.920192,78.033173,84.787500,86.130288,NaN,74.893269,NaN,85.262019,85.887981,...,-2.019231,3.00,-12.115385,17.40,-11.509615,5.12,-1.898077,17.56,-17.163462,11.12


In [164]:
def time_to_intercept(player, target, params):
    """
    Calcule le temps minimal pour que `player` atteigne `target`, en tenant compte de :
    - projection de la vitesse sur la direction cible (direction),
    - accélération maximale (a_max),
    - vitesse maximale (v_max),
    - temps de réaction (reaction_time).
    player: dict avec keys 'x','y','vx','vy'
    target: tuple (tx, ty)
    params: dict contenant 'v_max', 'a_max', 'reaction_time'
    Retour: t_intercept (en secondes)
    """
    v_max = params['v_max']
    a_max = params['a_max']
    reaction_time = params['reaction_time']

    px, py = float(player['x']), float(player['y'])
    vx, vy = float(player.get('vx', 0.0)), float(player.get('vy', 0.0))
    tx, ty = float(target[0]), float(target[1])

    dx = tx - px
    dy = ty - py
    d = np.hypot(dx, dy)
    if d < 1e-6:
        return reaction_time  # déjà quasiment dessus

    # direction unitaire vers la cible
    ux, uy = dx / d, dy / d

    # projection de la vitesse sur la direction cible
    v_proj = vx * ux + vy * uy
    v_proj = max(0.0, v_proj)  # si va dans l'autre sens => 0 contribution immédiate

    # on suppose que dès la réaction, le joueur applique une accélération a_max dans la direction cible
    # avec vitesse initiale v_proj (projection). il ne peut dépasser v_max.
    # on calcule le temps nécessaire pour parcourir la distance d, après le temps de reaction_time.

    # CAS 1: si la vitesse initiale est déjà >= v_max (rare), on parcourt d à vitesse v_proj (coupée à v_max)
    v0 = min(v_proj, v_max)

    # Si acceleration nulle (a_max == 0) fallback
    if a_max <= 1e-6:
        motion_time = d / max(1e-6, v0 if v0 > 1e-6 else v_max)
        return reaction_time + motion_time

    # Temps pour accélérer de v0 à v_max
    if v0 < v_max:
        t_acc = (v_max - v0) / a_max
        # distance parcourue pendant accélération
        dist_acc = v0 * t_acc + 0.5 * a_max * t_acc**2
    else:
        t_acc = 0.0
        dist_acc = 0.0

    # Si la distance pour atteindre la cible est inférieure à dist_acc,
    # le joueur n'atteint pas v_max avant d'arriver. Résolution de la quadratique :
    # 0.5 * a * t^2 + v0 * t - d = 0  (t >= 0)
    if dist_acc >= d and a_max > 0:
        # solve 0.5 a t^2 + v0 t - d = 0
        A = 0.5 * a_max
        B = v0
        C = -d
        disc = B*B - 4*A*C
        if disc < 0:
            disc = 0.0
        t_motion = (-B + np.sqrt(disc)) / (2*A)
        motion_time = t_motion
    else:
        # il atteint v_max (peut etre), et parcours le reste avec v_max
        dist_rem = max(0.0, d - dist_acc)
        if v_max <= 1e-6:
            # degenerate fallback
            motion_time = t_acc + dist_rem / (v0 + 1e-6)
        else:
            motion_time = t_acc + dist_rem / v_max

    return reaction_time + motion_time

In [165]:
def calculate_pitch_control_at_target_full(target_pos, attacking_players, defending_players, params):
    """
    Version complète inspirée de 'Physics of Possession'.
    - compute t_i for chaque joueur (avec direction + accel + v_max + reaction)
    - discrétise le temps t = 0..time_max par pas dt
    - à chaque pas t, calcule des poids w_i(t) = exp(-lambda*(t - t_i)) pour t >= t_i (0 sinon)
    - normalise les poids entre tous les joueurs encore 'disponibles' pour l'instant t
    - distribue la probabilité d'interception à ce pas à chaque joueur proportionnellement
    - accumule la probabilité par équipe
    Retourne {'P_att':..., 'P_def':...}
    """
    # paramètres
    lambda_param = params.get('lambda_param', 3.0)   # vitesse de décroissance des poids temporels
    time_max = params.get('time_max', 6.0)           # horizon (s)
    dt = params.get('dt', 0.02)                      # pas d'intégration (s)
    goalkeeper_ids = params.get('goalkeeper_ids', set())  # si on veut traiter gardien différemment (option)

    # filtrer joueurs valides
    att_valid = [p for p in attacking_players if (p is not None and not np.isnan(p['x']) and not np.isnan(p['y']))]
    def_valid = [p for p in defending_players if (p is not None and not np.isnan(p['x']) and not np.isnan(p['y']))]

    # cas limites
    if len(att_valid) == 0 and len(def_valid) == 0:
        return {'P_att': 0.5, 'P_def': 0.5}
    if len(att_valid) == 0:
        return {'P_att': 0.0, 'P_def': 1.0}
    if len(def_valid) == 0:
        return {'P_att': 1.0, 'P_def': 0.0}

    # calcul des temps d'arrivée moyens (ti) pour chaque joueur
    all_players = []
    for p in att_valid:
        ti = time_to_intercept(p, target_pos, params)
        all_players.append({'team': 'att', 'player': p, 't_mean': ti})
    for p in def_valid:
        ti = time_to_intercept(p, target_pos, params)
        all_players.append({'team': 'def', 'player': p, 't_mean': ti})

    # tri optionnel (non nécessaire), on fera l'intégration
    # discrétisation temporelle
    t_arr = np.arange(0.0, time_max + dt/2, dt)

    # probabilités accumulées par joueur et par équipe
    prob_player = np.zeros(len(all_players))  # prob mass accumulée par joueur
    prob_team_att = 0.0
    prob_team_def = 0.0

    # on suit la probabilité totale déjà capturée (pour arrêter si proche de 1)
    captured_total = 0.0

    # pour chaque pas de temps on calcule les poids et on distribue la petite masse de probabilité
    
    # intuition : plus le joueur arrive tôt (ti), mieux c'est, car on sommera potentiellement plus de poids (selon le pas dt choisi) sur des t >= ti
    # intuition : mais plus on calcule les poids sur des t qui s'éloigne de ti, moins le poids est grand (exponentielle décroissante), donc pour des t > aux ti de plusieurs joueurs, l'avantage d'arriver en premier diminue.
    for k, t in enumerate(t_arr):
        # poids pour chaque joueur à ce temps (0 si t < t_mean)
        weights = np.array([np.exp(-lambda_param * max(0.0, (t - pl['t_mean']))) if t >= pl['t_mean'] else 0.0 for pl in all_players])

        sumw = np.sum(weights)
        if sumw <= 0:
            continue  # rien à distribuer à ce pas

        # la 'masse' de probabilité disponible à ce pas : la portion qui n'a pas encore été capturée
        mass = (1.0 - captured_total)
        # on peut pondérer mass par dt (pour faire une intégration), en effet on veut approx integral
        # méthode simple : attribuer mass * (dt / time_max) ou mieux : suppose que distribution se normalise
        # Ici on distribue mass * (dt / (remaining_time_estimate)) - méthode pratique :
        # mais plus simple et stable : on suppose qu'on distribue mass * (lambda_norm * dt)
        # où lambda_norm est relative à la somme des exponentielles (on s'assure que intégré sur t donne 1).
        # pour stabilité on défini une petite fraction par pas :
        small_mass = mass * (1 - np.exp(-lambda_param * dt))  # heuristique -> approx mass * lambda*dt for small dt

        # distribuer small_mass proportionnellement aux weights
        probs_at_t = small_mass * (weights / sumw)

        # ajouter à prob_player & aux équipes
        prob_player += probs_at_t
        # mettre à jour captured_total
        captured_total = np.sum(prob_player)
        if captured_total >= 0.9999:
            break

    # somme par équipe
    for idx, pl in enumerate(all_players):
        if pl['team'] == 'att':
            prob_team_att += prob_player[idx]
        else:
            prob_team_def += prob_player[idx]

    # normalisation (sécurité numérique)
    total = prob_team_att + prob_team_def
    if total <= 0:
        return {'P_att': 0.5, 'P_def': 0.5}
    P_att = prob_team_att / total
    P_def = prob_team_def / total

    return {'P_att': float(P_att), 'P_def': float(P_def)}

In [193]:
# Créer les datasets équipe
home_tr, home_players = build_team_tracking(df_normed, 'home')
away_tr, away_players = build_team_tracking(df_normed, 'away')

# Calculer vitesses
home_tr = calculate_player_velocities(home_tr, home_players)
away_tr = calculate_player_velocities(away_tr, away_players)

In [202]:
# Calculer pitch control pour un point donné
frame = 28
att = get_players_state(home_tr, frame, home_players)
def_ = get_players_state(away_tr, frame, away_players)
target = (50, 50)  # exemple centre du terrain

pc = calculate_pitch_control_at_target_full(target, att, def_, params)
print(pc)

{'P_att': 0.3936309661568803, 'P_def': 0.6063690338431197}


In [166]:
def get_players_state(tracking, frame, players):
    """
    Récupère positions et vitesses des joueurs pour une frame donnée, en ignorant les NaN.
    Retourne liste de dicts [{'id', 'x', 'y', 'vx', 'vy'}, ...]
    """
    row = tracking.loc[tracking['frame'] == frame]
    if row.empty:
        return []
    row = row.iloc[0]
    player_states = []
    for pid in players:
        x = row.get(f'x_rescaled_{pid}', np.nan)
        y = row.get(f'y_rescaled_{pid}', np.nan)
        vx = row.get(f'vx_{pid}', np.nan)
        vy = row.get(f'vy_{pid}', np.nan)
        if pd.isna(x) or pd.isna(y):
            continue
        player_states.append({
            'id': pid,
            'x': float(x),
            'y': float(y),
            'vx': 0.0 if pd.isna(vx) else float(vx),
            'vy': 0.0 if pd.isna(vy) else float(vy)
        })
    return player_states

In [167]:
def calculate_individual_pitch_control_all_pitch(
    player_id,
    frame,
    home_tracking,
    away_tracking,
    home_players,
    away_players,
    params,
    pitch_length=105,
    pitch_width=68,
    resolution=3,
    half_pitch=False
):
    """
    Calcule le Pitch Control d’un joueur spécifique (player_id)
    contre tous les adversaires, à un instant donné (frame).
    
    Si half_pitch=True, le calcul ne concerne que la moitié droite du terrain.
    """

    # ---- 1. Identifier les équipes et joueurs ----
    if player_id in home_players:
        att_players = get_players_state(home_tracking, frame, [player_id])
        def_players = get_players_state(away_tracking, frame, away_players)
    elif player_id in away_players:
        att_players = get_players_state(away_tracking, frame, [player_id])
        def_players = get_players_state(home_tracking, frame, home_players)
    else:
        raise ValueError(f"player_id {player_id} not found in tracking data")

    # ---- 2. Grille spatiale ----
    n_grid_x = int(50 * resolution)
    n_grid_y = int(32 * resolution)

    if half_pitch:
        x_min, x_max = pitch_length / 2, pitch_length
    else:
        x_min, x_max = 0, pitch_length

    x = np.linspace(x_min, x_max, n_grid_x)
    y = np.linspace(0, pitch_width, n_grid_y)
    P_player_grid = np.zeros((len(y), len(x)))

    # ---- 3. Calcul du pitch control sur chaque cellule ----
    for ix, xx in enumerate(x):
        for iy, yy in enumerate(y):
            try:
                p = calculate_pitch_control_at_target_full(
                    (xx, yy), att_players, def_players, params
                )
                P_player_grid[iy, ix] = (
                    p["P_att"] if not np.isnan(p["P_att"]) else 0.5
                )
            except Exception:
                P_player_grid[iy, ix] = 0.5

    P_player_grid[P_player_grid == 0.5] = 0

    return P_player_grid

In [168]:
def calculate_defensive_lines(
    frame,
    away_tracking,
    away_players,
    n_clusters_options=(2, 3)
):

    # 1️⃣ Extraire les positions (x, y) des joueurs adverses pour la frame
    positions = []
    player_ids = []
    row = away_tracking.loc[away_tracking['frame'] == frame]
    for pid in away_players:
        if f"x_rescaled{pid}" in row.columns and f"y_rescaled_{pid}" in row.columns:
            x = row[f"x_rescaled_{pid}"].values[0]
            y = row[f"y_rescaled_{pid}"].values[0]
            if not np.isnan(x) and not np.isnan(y):
                positions.append([x, y])
                player_ids.append(pid)

    if len(positions) < 4:
        print(f"Frame {frame}: pas assez de joueurs adverses pour former des lignes défensives.")
        return None

    positions = np.array(positions)

    # Identifier et retirer le gardien (le joueur avec le x le plus faible)
    gk_index = np.argmax(positions[:, 0])  # le plus proche du but (à gauche)
    positions = np.delete(positions, gk_index, axis=0)
    player_ids = np.delete(player_ids, gk_index)

    # 2️⃣ Tester plusieurs nombres de clusters (2 ou 3 typiquement)
    best_model = None
    best_score = np.inf

    for n_clusters in n_clusters_options:
        model = AgglomerativeClustering(n_clusters=n_clusters)
        labels = model.fit_predict(positions[:, [0]])  # on clusterise selon x uniquement
        # mesurer la cohésion intra-cluster (écart-type moyen)
        intra_var = np.mean([np.std(positions[labels == k, 0]) for k in range(n_clusters)])
        if intra_var < best_score:
            best_score = intra_var
            best_model = model

    # 3️⃣ Récupérer les clusters finaux
    labels = best_model.fit_predict(positions[:, [0]])
    n_clusters = len(np.unique(labels))

    # 4️⃣ Calculer la ligne moyenne par cluster
    line_means = []
    for k in range(n_clusters):
        cluster_pts = positions[labels == k]
        x_mean = np.mean(cluster_pts[:, 0])
        line_means.append(x_mean)

    # Trier les lignes de la plus proche du but à la plus éloignée
    line_means = sorted(line_means)

    return line_means

In [173]:
def extract_player_context(
    frame: int,
    player_id: str,
    home_tr: pd.DataFrame,
    away_tr: pd.DataFrame,
    home_players: list,
    away_players: list
):
    """
    Extrait les informations de contexte d’un joueur à une frame donnée :
    - position du joueur
    - position du ballon
    - en possession du ballon (True/False)
    - distance au ballon
    - distance au coéquipier le plus proche
    - distance au joueur adverse le plus proche
    """

    # Déterminer si le joueur est à domicile ou à l'extérieur
    if player_id in home_players:
        team_tr, opp_tr = home_tr, away_tr
        teammates = [p for p in home_players if p != player_id]
        opponents = away_players
    elif player_id in away_players:
        team_tr, opp_tr = away_tr, home_tr
        teammates = [p for p in away_players if p != player_id]
        opponents = home_players
    else:
        raise ValueError(f"Le player_id {player_id} n'appartient à aucune des deux équipes.")

    # Extraire la ligne correspondant à la frame
    row_team = team_tr.loc[team_tr["frame"] == frame]
    row_opp = opp_tr.loc[opp_tr["frame"] == frame]
    if row_team.empty or row_opp.empty:
        raise ValueError(f"La frame {frame} est absente du tracking fourni.")

    # ---- 1️⃣ Position du joueur ----
    player_pos = np.array([
        row_team[f"x_rescaled_{player_id}"].values[0],
        row_team[f"y_rescaled_{player_id}"].values[0]
    ])

    # ---- 2️⃣ Position du ballon ----
    ball_pos = np.array([
        row_team["ball_x"].values[0],
        row_team["ball_y"].values[0]
    ])

    # ---- 3️⃣ Est-ce que le joueur est en possession ? ----
    player_in_possession_id = row_team["player_id_in_possession"].values[0]
    in_possession = player_in_possession_id == player_id

    # ---- 4️⃣ Distance au ballon ----
    distance_to_ball = np.linalg.norm(player_pos - ball_pos)

    # ---- 5️⃣ Distance au coéquipier le plus proche ----
    teammate_positions = []
    for pid in teammates:
        x, y = row_team[f"x_rescaled_{pid}"].values[0], row_team[f"y_rescaled_{pid}"].values[0]
        if not np.isnan(x) and not np.isnan(y):
            teammate_positions.append([x, y])

    if teammate_positions:
        teammate_positions = np.array(teammate_positions)
        distance_to_nearest_teammate = np.min(np.linalg.norm(teammate_positions - player_pos, axis=1))
    else:
        distance_to_nearest_teammate = np.nan

    # ---- 6️⃣ Distance au joueur adverse le plus proche ----
    opponent_positions = []
    for pid in opponents:
        x, y = row_opp[f"x_rescaled_{pid}"].values[0], row_opp[f"y_rescaled_{pid}"].values[0]
        if not np.isnan(x) and not np.isnan(y):
            opponent_positions.append([x, y])

    if opponent_positions:
        opponent_positions = np.array(opponent_positions)
        distance_to_nearest_opponent = np.min(np.linalg.norm(opponent_positions - player_pos, axis=1))
    else:
        distance_to_nearest_opponent = np.nan

    # ---- Résumé ----
    return {
        "frame": frame,
        "player_id": player_id,
        "player_position": tuple(player_pos),
        "ball_position": tuple(ball_pos),
        "in_possession": in_possession,
        "distance_to_ball": distance_to_ball,
        "distance_to_nearest_teammate": distance_to_nearest_teammate,
        "distance_to_nearest_opponent": distance_to_nearest_opponent
    }

In [174]:
def extract_player_pitch_control_and_contextual(
    player_id,
    frame,
    home_tracking,
    away_tracking,
    home_players,
    away_players,
    params,
    pitch_length=105,
    pitch_width=68,
    resolution=1,
    half_pitch=True
):

    # ---- 1. Calcul du pitch control individuel ----
    P_player_grid = calculate_individual_pitch_control_all_pitch(
        player_id,
        frame,
        home_tracking,
        away_tracking,
        home_players,
        away_players,
        params,
        pitch_length,
        pitch_width,
        resolution,
        half_pitch
    )

    # ---- 2. Identify defensive lines ----
    line_means = calculate_defensive_lines(
        frame,
        away_tracking,
        away_players,
        n_clusters_options=(2, 3)
    )

    # ---- 3. Extraction du contexte du joueur ----
    player_context = extract_player_context(
        frame,
        player_id,
        home_tracking,
        away_tracking,
        home_players,
        away_players
    )

    player_frame_info = {
        "pitch_control_map": P_player_grid,
        "defensive_lines": line_means
    }

    player_frame_info.update(player_context)

    return player_frame_info

In [184]:
def extract_all_for_position_across_match(
    match_id: str,
    player_position: str,
    params: dict,
    resolution: int = 1,
    pitch_length: float = 105,
    pitch_width: float = 68
):
    """
    Calcule les cartes de pitch control et le contexte joueur pour une position donnée,
    sur tous les fichiers de tracking d’un répertoire de possessions.

    Retourne une liste de dictionnaires contenant :
      - frame
      - pitch_control_map
      - defensive_lines
      - player_id
      - player_position (x, y)
      - ball_position (x, y)
      - in_possession (True/False)
      - distance_to_ball
      - distance_to_nearest_teammate
      - distance_to_nearest_opponent
    """

    all_results = []

    # ---- 2️⃣ Charger le dataframe ----
    df = pd.read_parquet(f"{PROCESSED_DIR}/{match_id}_normed.parquet", storage_options={"token": "google_default"})

    # ---- 3️⃣ Identifier les player_id correspondant à la position ----
    player_ids_by_team = get_player_id_position(df, player_position)
    home_team_id = df["home_team_id"].iloc[0]
    away_team_id = df["away_team_id"].iloc[0]
    home_player_id = player_ids_by_team[home_team_id]
    away_player_id = player_ids_by_team[away_team_id]

    # ---- 4️⃣ Construire les tracking datasets ----
    home_tr, home_players = build_team_tracking(df, "home")
    away_tr, away_players = build_team_tracking(df, "away")

    # ---- 5️⃣ Calculer les vitesses ----
    home_tr = calculate_player_velocities(home_tr, home_players)
    away_tr = calculate_player_velocities(away_tr, away_players)

    # ---- 6️⃣ Itérer sur les frames ----
    frames = list(df["frame"].unique())
    frames_test = frames[-2:]  # pour test rapide
    #frames_test = random.sample(frames, 50)
    for frame in tqdm(frames_test, desc=f"Match {match_id}", leave=False):

        # Identifier l’équipe en possession
        in_possession_team = df.loc[df["frame"] == frame, "team_in_possession"].values[0]

        # Déterminer le player_id du joueur concerné
        if in_possession_team == home_team_id:
            player_id = home_player_id
            if player_id is None:
                continue  # aucune correspondance -> on saute
        elif in_possession_team == away_team_id:
            player_id = away_player_id
            if player_id is None:
                continue
        else:
            continue  # frame invalide ou sans possession claire

        # ---- 7️⃣ Calcul complet pour la frame ----
        player_frame_info = extract_player_pitch_control_and_contextual(
            player_id=player_id,
            frame=frame,
            home_tracking=home_tr,
            away_tracking=away_tr,
            home_players=home_players,
            away_players=away_players,
            params=params,
            pitch_length=pitch_length,
            pitch_width=pitch_width,
            resolution=resolution,
            half_pitch=True
        )

        # ---- 8️⃣ Ajout d’informations globales ----
        player_frame_info.update({
            "frame": frame,
            "player_id": player_id,
            "player_position_role": player_position
        })

        all_results.append(player_frame_info)

    print(f"\n✅ Traitement terminé : {len(all_results)} frames analysées au total.")
    return all_results

In [185]:
%%time

results = extract_all_for_position_across_match(
        match_id=1886347,
        player_position="Center Forward",
        params=params,
        resolution=1
    )

   match_id                   time  frame  period  player_id  is_detected  \
0   1886347 0 days 00:00:01.800000     28       1      14736         True   
1   1886347 0 days 00:00:01.800000     28       1      23418         True   
2   1886347 0 days 00:00:01.800000     28       1      33697         True   
3   1886347 0 days 00:00:01.800000     28       1      38673         True   
4   1886347 0 days 00:00:01.800000     28       1      50951         True   

   is_ball      x     y  x_rescaled  ...  player_in_possession_id  \
0    False   9.73  6.44   62.323558  ...                   966120   
1    False   9.03 -2.78   61.616827  ...                   966120   
2    False  17.43 -3.57   70.097596  ...                   966120   
3    False   1.76  9.23   54.276923  ...                   966120   
4    False  -0.07 -8.13   52.429327  ...                   966120   

   team_in_possession  team_in_possession_shortname  attacking_side  \
0                1805                     Newcastle

Frame 58733: pas assez de joueurs adverses pour former des lignes défensives.
Frame 58907: pas assez de joueurs adverses pour former des lignes défensives.

✅ Traitement terminé : 2 frames analysées au total.
CPU times: user 240 ms, sys: 15.7 ms, total: 256 ms
Wall time: 2.06 s


In [186]:
results

[{'pitch_control_map': array([[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]),
  'defensive_lines': None,
  'frame': 58733,
  'player_id': 966120,
  'player_position': (nan, nan),
  'ball_position': (60.40528846153846, 27.429999999999993),
  'in_possession': False,
  'distance_to_ball': nan,
  'distance_to_nearest_teammate': nan,
  'distance_to_nearest_opponent': nan,
  'player_position_role': 'Center Forward'},
 {'pitch_control_map': array([[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]),
  'defensive_lines': None,
  'frame': 58907,
  'player_id': 38673,
  'player_position': (nan, nan),
  'ball_p